# Constructors

In [1]:
struct Foo
    bar
    baz
end

In [2]:
foo = Foo(1, 2)

Foo(1, 2)

In [3]:
foo.bar

1

In [4]:
foo.baz

2

## Outer Constructor Methods

In [5]:
Foo(x) = Foo(x, x)

Foo

In [6]:
Foo(1)

Foo(1, 1)

In [7]:
Foo() = Foo(0)

Foo

In [8]:
Foo()

Foo(0, 0)

## Inner Constructor Methods

In [9]:
struct OrderedPair
    x::Real
    y::Real
    OrderedPair(x,y) = x > y ? error("out of order") : new(x,y)
end

In [12]:
OrderedPair(1,2)

OrderedPair(1, 2)

In [13]:
OrderedPair(2,1)

ErrorException: [91mout of order[39m

In [18]:
struct Foo
    bar
    baz
    Foo(bar,baz) = new(bar,baz)
end

In [19]:
struct T1
    x::Int64
end

In [20]:
struct T2
    x::Int64
    T2(x) = new(x)
end

In [21]:
T1(1)

T1(1)

In [22]:
T2(1)

T2(1)

In [24]:
T1(1.0)

T1(1)

In [23]:
T2(1.0)

T2(1)

## Incomplete Initialization

In [1]:
# mutable struct SelfReferential
#     obj::SelfReferential
# end

In [2]:
# b = SelfReferential(a)

In [3]:
mutable struct SelfReferential
    obj::SelfReferential
    SelfReferential() = (x = new(); x.obj = x)
end

In [8]:
x = SelfReferential();

In [9]:
x == x

true

In [10]:
x === x.obj

true

In [12]:
x === x.obj.obj

true

In [13]:
mutable struct Incomplete
    data
    Incomplete() = new()
end

In [14]:
z = Incomplete()

Incomplete(#undef)

In [15]:
z.data

UndefRefError: [91mUndefRefError: access to undefined reference[39m

In [16]:
struct HasPlain
    n::Int
    HasPlain() = new()
end

In [17]:
HasPlain()

HasPlain(139661113913488)

In [19]:
mutable struct Lazy
    data
    Lazy(v) = complete_me(new(), v)
end

## Parametric Constructors

In [20]:
struct Point{T<:Real}
    x::T
    y::T
end

In [21]:
Point(1,2)

Point{Int64}(1, 2)

In [22]:
Point(1.0, 2.5)

Point{Float64}(1.0, 2.5)

In [23]:
Point(1, 2.5)

MethodError: [91mMethodError: no method matching Point(::Int64, ::Float64)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  Point(::T, [91m::T[39m) where T<:Real at In[20]:2[39m

In [24]:
Point{Int64}(1, 2)

Point{Int64}(1, 2)

In [25]:
Point{Int64}(1.0, 2.5)

InexactError: [91mInexactError: Int64(2.5)[39m

In [26]:
Point{Float64}(1.0, 2.5)

Point{Float64}(1.0, 2.5)

In [27]:
Point{Float64}(1,2)

Point{Float64}(1.0, 2.0)

In [29]:
struct Point{T<:Real}
    x::T
    y::T
    Point{T}(x,y) where {T<:Real} = new(x,y)
end

In [32]:
Point(x::Int64, y::Float64) = Point(convert(Float64,x),y)

Point

In [33]:
Point(1,2.5)

Point{Float64}(1.0, 2.5)

In [34]:
typeof(ans)

Point{Float64}

In [35]:
Point(1.5,2)

MethodError: [91mMethodError: no method matching Point(::Float64, ::Int64)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  Point(::T, [91m::T[39m) where T<:Real at In[20]:2[39m

In [36]:
Point(x::Real, y::Real) = Point(promote(x,y)...)

Point

In [37]:
Point(1.5,2)

Point{Float64}(1.5, 2.0)

In [38]:
Point(1,1//2)

Point{Rational{Int64}}(1//1, 1//2)

In [39]:
Point(1.0,1//2)

Point{Float64}(1.0, 0.5)

## Case Study:Rational

In [40]:
struct OurRational{T<:Integer} <: Real
    num::T
    den::T
    function OurRational{T}(num::T, den::T) where T<:Integer
        if num == 0 && den == 0
            error("invalid rational: 0//0")
        end
        g = gcd(den, num)
        num = div(num, g)
        den = div(den, g)
        new(num, den)
    end
end

In [41]:
OurRational(n::T, d::T) where {T<:Integer} = OurRational{T}(n,d)

OurRational

In [42]:
OurRational(n::Integer, d::Integer) = OurRational(promote(n,d)...)

OurRational

In [43]:
OurRational(n::Integer) = OurRational(n, one(n))

OurRational

In [44]:
⊘(n::Integer, d::Integer) = OurRational(n,d)

⊘ (generic function with 1 method)

In [45]:
⊘(x::OurRational, y::Integer) = x.num ⊘ (x.den*y)

⊘ (generic function with 2 methods)

In [46]:
⊘(x::Integer, y::OurRational) = (x*y.den) ⊘ y.num

⊘ (generic function with 3 methods)

In [47]:
⊘(x::Complex, y::Real) = complex(real(x) ⊘ y, imag(x) ⊘ y)

⊘ (generic function with 4 methods)

In [48]:
⊘(x::Real, y::Complex) = (x*y') ⊘ real(y*y')

⊘ (generic function with 5 methods)

In [49]:
function ⊘(x::Complex, y::Complex)
    xy = x*y'
    yy = real(y*y')
    complex(real(xy) ⊘ yy, imag(xy) ⊘ yy)
end

⊘ (generic function with 6 methods)

In [50]:
z = (1 + 2im) ⊘ (1 - 2im);

In [51]:
typeof(z)

Complex{OurRational{Int64}}

In [52]:
typeof(z) <: Complex{OurRational}

false

## Outer-only constructors

In [61]:
struct SummedArray{T<:Number,S<:Number}
    data::Vector{T}
    sum::S
end

In [62]:
struct SummedArray{T<:Number, S<:Number}
    data::Vector{T}
    sum::S
    function SummedArray(a::Vector{T}) where T
        S = widen(T)
        new{T,S}(a, sum(S, a))
    end
end